In [ ]:
!pip install requests

In [ ]:
!pip3 install beautifulsoup4

In [ ]:
# get data input for nasdaq
import pandas as pd
import os


dir_name= os.getcwd()+'/data/'

nasdaq_files=os.path.join(dir_name,'stock_ticker_datasets/nasdaq.csv')
nyse_files=os.path.join(dir_name,'stock_ticker_datasets/nyse.csv')


# access the data folder to get the list of tickers in nasdaq
nasdaq=pd.read_csv(nasdaq_files)

# access the data folder to get the list of tickers in nasdaq
nyse=pd.read_csv(nyse_files)

nasdaq['Symbol']=nasdaq['Symbol'].astype(str)
nasdaq_input=nasdaq['Symbol']

nyse['Symbol']=nyse['Symbol'].astype(str)
nyse_input=nyse['Symbol']

#chunk row size
n = 400

#split datasets into smaller sets to prevent being blocked from the API
nasdaq_df = [nasdaq_input[i:i+n] for i in range(0,nasdaq_input.shape[0],n)]
nyse_df = [nyse_input[i:i+n] for i in range(0,nyse_input.shape[0],n)]

# merge all the tickers in the US market
df=nasdaq_df+nyse_df
nasdaq.set_index("Symbol" , inplace=True)
nyse.set_index("Symbol" , inplace=True)




In [ ]:
from bs4 import BeautifulSoup
from urllib.request import Request
from urllib.request import urlopen
import csv
import datetime, time


news_tables = {}

# prefix url to access finviz.com
finviz_url = 'https://finviz.com/quote.ashx?t='

def get_finviz(ticker,day):
    try:
        url = finviz_url + ticker
        req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
        resp = urlopen(req) 
        html = BeautifulSoup(resp, features="lxml")
        # access the news stored under the <div id='news-tabel'>
        news_table = html.find(id='news-table')   
        news_tables[ticker] = news_table
        df = news_tables[ticker]
        path=os.path.join(dir_name,'data-news/data-finviz/'+'data-'+ticker+'-finviz.csv')
        # append the news into the corresponding ticker's csv file
        with open(path,'a') as f:
            print(ticker)
            writer = csv.writer(f)
            for info in df.findAll('tr'):
                # get the text of the financial headlines
                text=info.a.get_text()
                date_scrape= info.td.text.split()
                if(len(date_scrape)==1):
                    time=date_scrape[0]
                else:
                    date= date_scrape[0]
                    time=date_scrape[1]
                    news_time_str= date+" "+time;
                # convert the date into datetime object of 'YYYY-mm-dd' format
                date_time_obj = datetime.datetime.strptime(news_time_str, '%b-%d-%y %I:%M%p').strftime('%Y-%m-%d')
                if(datetime.datetime.now()-date_time_obj).days<=day:
                    writer.writerow([date_time_obj,text])

        
    except:
        print("error")
        pass



In [ ]:
    
# getting news data for tickers listed in nasdaq        
for tickers in nasdaq_df:
    for ticker in tickers:
        get_finviz(ticker,1)
        
# getting news data for tickers listed in nyse       
for tickers in df:
    for ticker in tickers:
        get_finviz(ticker,1)




In [ ]:
nyse_pd= pd.DataFrame()
nasdaq_pd = pd.DataFrame()


# merge all the news in nasdaq from finviz.com
for nas in nasdaq_df:
     for symbol in nas:
        try:

            df_news = pd.read_csv('/userhome/cs/wuxue/Sentiment_Analysis/data-news/data-finviz/'+'data-%s-finviz.csv'%symbol,names=['dates','news'])
            sector=nasdaq.loc[symbol]['Sector']
            ticker=[symbol for i in range(len(df_news))]
            Sector=[sector for i in range(len(df_news))]
            df_news['ticker']=ticker
            df_news['sector']=Sector
            nasdaq_pd=nasdaq_pd.append(df_news)  
            print(nasdaq_pd)
        except:
            print('no such file exist')
            pass

# merge all the news in nasdaq from finviz.com
for ny in nyse_df:
     for symbol in ny:
        try:
            path=os.path.join(dir_name,'data-news/data-finviz/'+'data-'+symbol+'-finviz.csv')
            df_news = pd.read_csv(path,names=['dates','news'])
            sector=nyse.loc[symbol]['Sector']
            ticker=[symbol for i in range(len(df_news))]
            Sector=[sector for i in range(len(df_news))]
            df_news['ticker']=ticker
            df_news['sector']=Sector
            nyse_pd=nyse_pd.append(df_news)  
            print(nyse_pd)
        except:
            print('no such file exist')
            pass


# store as csv file in the following names
nasdaq_pd.to_csv('nasdaq_no_labelled_news.csv')
nyse_pd.to_csv('nyse_no_labelled_news.csv')

